In [ ]:
#Requires to have geopandas installed
#Installation https://geopandas.org/getting_started/install.html

import matplotlib.pyplot as plt 
import geopandas as gpd
import geoplot
import pandas as pd
import fiona #;help(fiona.open)
import requests
from pandas import json_normalize
import requests # convert json into dataframe
#For OSM 
import pyrosm
import numpy as np
from pyrosm import OSM, get_data

# Africa shape data

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#world

In [ ]:
saudi = world.query('name == "Saudi Arabia"')
yemen = world.query('name == "Yemen"')
iraq = world.query('name == "Iraq"')
oman = world.query('name == "Oman"')
uae = world.query('name == "United Arab Emirates"')
jordan = world.query('name == "Jordan"')
palestine = world.query('name == "Palestine"')
israel = world.query('name == "Israel"')
qatar = world.query('name == "Qatar"')
kuwait = world.query('name == "Kuwait"')

mena = saudi.append(yemen).append(iraq).append(oman).append(uae).append(jordan).append(palestine).append(israel).append(qatar).append(kuwait)

In [ ]:
africa = world.query('continent == "Africa"').append(mena)
#africa.head()

In [ ]:
#africa.plot()

In [ ]:
nigeriaplot = world.query('name == "Nigeria"')

# Power data

In [ ]:
import os
import sys

In [ ]:
pathg = os.path.realpath("data")+'/africa_all_generators.geojson'
paths = os.path.realpath("data")+'/africa_all_substations.geojson'
pathl = os.path.realpath("data")+'/africa_all_lines.geojson'

generators = gpd.read_file(pathg)
points = gpd.read_file(paths)
lines = gpd.read_file(pathl)

In [ ]:
points

In [ ]:
offset = 0.001
bbox = points.bounds + [-offset, -offset, offset, offset]
hits = bbox.apply(lambda row: list(lines.sindex.intersection(row)), axis=1)

In [ ]:
bbox

In [ ]:
lines

In [ ]:
display(hits)

In [ ]:
tmp = pd.DataFrame({
    # index of points table
    "pt_idx": np.repeat(hits.index, hits.apply(len)),
    # ordinal position of line - access via iloc later
    "line_i": np.concatenate(hits.values)
})

In [ ]:
tmp

In [ ]:
# Join back to the lines on line_i; we use reset_index() to 
# give us the ordinal position of each line
tmp = tmp.join(lines.reset_index(drop=True), on="line_i")
# Join back to the original points to get their geometry
# rename the point geometry as "point"
tmp = tmp.join(points.geometry.rename("point"), on="pt_idx")
# Convert back to a GeoDataFrame, so we can do spatial ops
tmp = gpd.GeoDataFrame(tmp, geometry="geometry", crs=points.crs)

In [ ]:
tmp["snap_dist"] = tmp.geometry.distance(gpd.GeoSeries(tmp.point))

In [ ]:
tolerance = 0.1 # 10km
# Discard any lines that are greater than tolerance from points
tmp = tmp.loc[tmp.snap_dist <= tolerance]
# Sort on ascending snap distance, so that closest goes to top
tmp = tmp.sort_values(by=["snap_dist"])



In [ ]:
# group by the index of the points and take the first, which is the
# closest line 
closest = tmp.groupby("pt_idx").first()
# construct a GeoDataFrame of the closest lines
closest = gpd.GeoDataFrame(closest, geometry="geometry")

In [ ]:
# Position of nearest point from start of the line
pos = closest.geometry.project(gpd.GeoSeries(closest.point))
# Get new point location geometry
new_pts = closest.geometry.interpolate(pos)

In [ ]:
#Identify the columns we want to copy from the closest line to the point, such as a line ID.
line_columns = 'line_i'
# Create a new GeoDataFrame from the columns from the closest line and new point geometries (which will be called "geometries")
snapped = gpd.GeoDataFrame(
closest[line_columns],geometry=new_pts)

# Join back to the original points:
updated_points = points.drop(columns=["geometry"]).join(snapped)
# You may want to drop any that didn't snap, if so:
updated_points = updated_points.dropna(subset=["geometry"])

In [ ]:
updated_points.to_file('./updates_points.geojson', driver="GeoJSON")  # Generate GeoJson

# Map

In [ ]:
ax = geoplot.polyplot(nigeriaplot ,edgecolor='grey', alpha = 0.5, figsize=(20, 20))

points.loc[points['Country'] == 'nigeria'].plot(figsize=(20, 20),markersize=10, color='orange',alpha = 0.3, ax=ax)
updated_points.loc[updated_points['Country'] == 'nigeria'].plot(figsize=(20, 20),markersize=10, color='green',alpha = 0.3, ax=ax)
generators.loc[generators['Country'] == 'nigeria'].plot(figsize=(20, 20),markersize=25, color='blue',alpha = 0.3, ax=ax)
lines.loc[lines['Country'] == 'nigeria'].plot(figsize=(20, 20),markersize=25, color='red',alpha = 0.3, ax=ax)


#plt.savefig('africa_transmission_and substations_110.png')